## MF731 Homework 4
### Edited by Xuyang Liu

In [1]:
import pandas as pd
import numpy as np
from scipy.stats import norm

### Stress Test for a Market-Cap Weighted Portfolio of Microsoft and Apple Stocks.

In [2]:
# pre-prepare the data
data = pd.read_csv("MSFT_AAPL_Log_Returns.csv",header=None,index_col=0)
data.index.name="Date"
data = data.rename(columns={1: 'MSFT', 2: 'AAPL'})

##### (1) Estimate mu and sigma using EWMA

In [3]:
lamda = 0.97
theta = 0.97
M = 100

logret = data[M:]
mu0 = data[:M].mean()
sig0 = data[:M].cov()
logmat = np.matrix(logret)

# Start EWMA algorithem
mu = np.matrix(mu0).T
sig = np.matrix(sig0)
for i in range(0,len(logret)):
    sig = theta * sig + (1-theta)*(logmat[i].T-mu)*(logmat[i]-mu.T)
    mu = lamda * mu + (1-lamda)*logmat[i].T

In [4]:
print("the Mean is about: ",mu.A1)
print("the Sigma is about: ",sig)

the Mean is about:  [0.00125025 0.00106069]
the Sigma is about:  [[1.29896289e-04 3.30063144e-05]
 [3.30063144e-05 1.51851733e-04]]


##### (2) Estimate VaR

In [5]:
Mhat = 50000
alpha = 0.95
K = 10
MSFT_cap = 448.77
AAPL_cap = 575.11
cap = 1000000

w = np.matrix([MSFT_cap/(MSFT_cap + AAPL_cap),AAPL_cap/(MSFT_cap + AAPL_cap)]).T
VaR1 = (-w.T * mu + np.sqrt(w.T*sig*w)*norm.ppf(alpha) ) * cap
VaR1[0,0]

14383.846180380484

In [6]:
VaRK = K**0.5 * VaR1
VaRK[0,0]

45485.71544351548

In [7]:
cap_change = 3 * VaRK
cap_change[0,0]

136457.14633054644

In [8]:
sig

matrix([[1.29896289e-04, 3.30063144e-05],
        [3.30063144e-05, 1.51851733e-04]])

##### (3) Shock the system by assuming a large negative return for Apple

In [9]:
# from the sigma matrix, we can get correlation rho
rho = ((sig[0,1] * sig[1,0])/ (sig[0,0] * sig[1,1]))**0.5
# (i)
x2 = mu[1,0] - 5 * sig[1,1]**0.5
# (ii)
mux1 = mu[0,0] + rho * sig[0,0]**0.5 / sig[1,1]**0.5 * (x2 - mu[1,0])
varx1 = sig[0,0] * (1-rho**2)

x1 = mux1 - 5 * varx1**0.5 

In [10]:
# (iii)
def update_EWMA(mu,sig,x1,x2):
    new_sig = lamda*sig + (1-lamda)*(np.matrix([x1,x2]).T-mu)*(np.matrix([x1,x2])-mu.T)
    new_mu = theta*mu + (1-theta)*np.matrix([x1,x2]).T
    
    return new_sig, new_mu

newsig,newmu = update_EWMA(mu,sig,x1,x2)
newsig,newmu

(matrix([[ 1.38081456e-04, -5.07849498e-06],
         [-5.07849498e-06,  2.61184981e-04]]),
 matrix([[ 0.0018523 ],
         [-0.00078773]]))

In [19]:
def simulations(newmu,newsig,Mhat,K):  
    loss_sim = []
    for i in range(Mhat):
        
        #x1 = mux1 + varx1**0.5 * np.random.normal(0,1)
        #sim_sig,sim_mu = update_EWMA(mu,sig,x1,x2)
        
        sim_mu = newmu
        sim_sig = newsig
        x1add = 0
        x2add = 0
        for i in range(K):
            sim_x1,sim_x2 = np.random.multivariate_normal(sim_mu.A1, sim_sig)
            sim_sig = lamda*sim_sig + (1-lamda)*(np.matrix([sim_x1,sim_x2]).T-sim_mu)*(np.matrix([sim_x1,sim_x2])-sim_mu.T)
            sim_mu = theta*sim_mu + (1-theta)*np.matrix([sim_x1,sim_x2]).T
            x1add += sim_x1
            x2add += sim_x2
        loss = -(w[0,0] * x1add + w[1,0]*x2add) * cap
        loss_sim.append(loss)
    
    return loss_sim 
    

In [20]:
Loss = simulations(newmu,newsig,Mhat,K)

In [21]:
ave_kday_loss = np.mean(Loss)
kday_var = np.quantile(Loss, alpha, interpolation="higher")
ave_kday_loss, kday_var

(-3519.6145730098688, 56952.66617211786)

In [22]:
u1 = VaRK.A1[0]
u2 = cap_change.A1[0]
Loss = np.array(Loss)
exceed_1 = len(Loss[Loss>u1])/len(Loss)*100
exceed_2 = len(Loss[Loss>u2])/len(Loss)*100
exceed_1, exceed_2

(8.984, 0.018000000000000002)